# Using distorter
 
 The concept of cation mutation (ion substitution) to tune the properties of compounds has been around at least [since the 1950s](https://www.sciencedirect.com/science/article/pii/0022369758900507) and has proved a useful concept in [more recent studies](https://journals.aps.org/prb/abstract/10.1103/PhysRevB.79.165211).

![](cation_mutation.png)

This example generates all possible (symmetry inequivalent) substitutions of Sr on Ba sites in the cubic perovskite
BaTiO3; single and double substitutions.
The distorter module uses the [ASE Python library](https://wiki.fysik.dtu.dk/ase/) to achieve this.

In [1]:
import smact.builder as builder
import smact.distorter as distort

In [2]:
def pretty_print_atoms(atoms, linewrap=15):
    entries = [
        "{0:5.3f} {1:5.3f} {2:5.3f} {symbol}".format(*position, symbol=symbol)
        for position, symbol in zip(atoms.get_positions(), atoms.get_chemical_symbols())
    ]

    for output_i in range(linewrap):
        line = ""
        for i, entry in enumerate(entries):
            if output_i == i % linewrap:
                line = line + entry + "\t"
        print(line)


# Build the input
smact_lattice, test_case = builder.cubic_perovskite(
    ["Ba", "Ti", "O"], repetitions=[2, 2, 2]
)

hlinewidth = 68


print("-" * hlinewidth)
print("Original coordinates: ")
pretty_print_atoms(test_case)
print("-" * hlinewidth)

# Do the single substitution first, it is trivial as all Ba
#  sites are equivalent we will choose the first Ba
subs_site = [0.0, 0.0, 0.0]
single_substitution = distort.make_substitution(test_case, subs_site, "Sr")
print("Single: ")
pretty_print_atoms(single_substitution)

# Build a sub-lattice you wish to disorder [test case do the Ba sub-lattice]
sub_lattice = distort.build_sub_lattice(single_substitution, "Ba")

# Enumerate the inequivalent sites
inequivalent_sites = distort.get_inequivalent_sites(sub_lattice, single_substitution)

# Replace Ba at inequivalent sites with Sr
for i, inequivalent_site in enumerate(inequivalent_sites):
    print("-" * hlinewidth)
    print(f"Substituted coordinates {i}")
    distorted = distort.make_substitution(single_substitution, inequivalent_site, "Sr")
    pretty_print_atoms(distorted)

print("=" * hlinewidth)

--------------------------------------------------------------------
Original coordinates: 
0.000 0.000 0.000 Ba	0.000 6.000 6.000 Ba	6.000 6.000 0.000 Ba	
3.000 3.000 3.000 Ti	3.000 9.000 9.000 Ti	9.000 9.000 3.000 Ti	
3.000 3.000 0.000 O	3.000 9.000 6.000 O	9.000 9.000 0.000 O	
0.000 3.000 3.000 O	0.000 9.000 9.000 O	6.000 9.000 3.000 O	
3.000 0.000 3.000 O	3.000 6.000 9.000 O	9.000 6.000 3.000 O	
0.000 0.000 6.000 Ba	6.000 0.000 0.000 Ba	6.000 6.000 6.000 Ba	
3.000 3.000 9.000 Ti	9.000 3.000 3.000 Ti	9.000 9.000 9.000 Ti	
3.000 3.000 6.000 O	9.000 3.000 0.000 O	9.000 9.000 6.000 O	
0.000 3.000 9.000 O	6.000 3.000 3.000 O	6.000 9.000 9.000 O	
3.000 0.000 9.000 O	9.000 0.000 3.000 O	9.000 6.000 9.000 O	
0.000 6.000 0.000 Ba	6.000 0.000 6.000 Ba	
3.000 9.000 3.000 Ti	9.000 3.000 9.000 Ti	
3.000 9.000 0.000 O	9.000 3.000 6.000 O	
0.000 9.000 3.000 O	6.000 3.000 9.000 O	
3.000 6.000 3.000 O	9.000 0.000 9.000 O	
--------------------------------------------------------------------
Single: 

In [11]:
# The distorted objects are of the ase atoms class,
# so can be written out to a crystal structure file like a cif
distorted.write("example_output.cif")